Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [121]:
#imports
import numpy as np
from copy import deepcopy
from random import random, randint

## TEST PROBLEMS

In [122]:
# # Problem 1:
# rng = np.random.default_rng(seed=42)
# NUM_KNAPSACKS = 3
# NUM_ITEMS = 20
# NUM_DIMENSIONS = 2
# VALUES = rng.integers(0, 100, size=NUM_ITEMS)
# WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
# CONSTRAINTS = rng.integers(
#     0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
# )

In [123]:
# # Problem 2:
# rng = np.random.default_rng(seed=42)
# NUM_KNAPSACKS = 10
# NUM_ITEMS = 100
# NUM_DIMENSIONS = 10
# VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
# WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
# CONSTRAINTS = rng.integers(
#     1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, 
#     size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
# )

In [124]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, 
    size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [125]:
#solution check
def check_sol(solution):
    valid = True
    score = 0
    cost = 0.0
    for k in range(NUM_KNAPSACKS):
        assigned_indices = np.where(solution[0] == k)[0]        #for each knap. it takes the indices of the items assigned to it
        if assigned_indices.size > 0:
            weights = WEIGHTS[assigned_indices].sum(axis=0)     #computes the combined weights of all items in the current knap along each dim.(NUM_DIM,)
            value = VALUES[assigned_indices].sum()              #computes the combined value of all items in the current knap. (scalar)
        else:
            weights = np.zeros(NUM_DIMENSIONS)
            value = 0
        violations = np.maximum(0, weights - CONSTRAINTS[k])    #computes the total number of contraints violations for each dim. for the current knap. (NUM_DIM,)
        if np.any(violations > 0):                              #if it finds a single violation, the solution is not valid
            valid = False
        cost += np.sum(violations / (CONSTRAINTS[k] + 1e-6))    #normalized cost for knap. (based on the capacity along each dim.)
        score += value
    return valid, score, cost

In [126]:
#initial solution
'''
I decided to represent the solutions with an array of NUM_ITEMS elements, to make it simpler
example:    [ 0, 1, 0, -1, 2, ... -1]
            means that:
                - item 0 is in knap. 0
                - item 1 is in knap. 1
                - item 2 is in knap. 0
                - item 3 isn't in any knap.
                - item 4 is in knap. 2
                - item NUM_ITEMS-1 isn't in any knap.
I used this greedy algo. (puts an item in the best knap. at the moment) because starting from a completely randomic solution
didn't allow me to find a valid one in a reasonable time 
'''
def initial_solution():
    solution = -1 * np.ones((1, NUM_ITEMS), dtype=int)      #starting from an array of -1s
    for i in range(NUM_ITEMS):
        best_k = -1                                         #it will store the knapsack index where item i should go
        min_overload = np.inf                               #keeps track of the smallest tot. oversload across all knap., in order to find the best for item i
        for k in range(NUM_KNAPSACKS):
            assigned_indices = np.where(solution[0] == k)[0]
            current_weights = WEIGHTS[assigned_indices].sum(axis=0) if assigned_indices.size > 0 else np.zeros(NUM_DIMENSIONS)  #sum of the weights of a knap. for each dim.
            potential_weights = current_weights + WEIGHTS[i]                                                                    #hypotetical sum of weights if I added i to the knap.
            overload = np.sum(np.maximum(0, potential_weights - CONSTRAINTS[k]) / (CONSTRAINTS[k] + 1e-6))                      #measures how badly the knap. would be violated if I added i to it (ChatGPT helped with it)
            if overload < min_overload:                     #keeps track of the best knap. for item i
                min_overload = overload
                best_k = k
        if min_overload == 0:
            solution[0, i] = best_k
        else:
            solution[0, i] = -1
    return solution

In [127]:
#tweak functions
'''
I had to include tabu search to the tweak I use until I find a valid solution. I tried without tabu search but it
couldn't find a valid solution in a reasonable time
'''

TABU_SIZE = 100
tabu_list = []

#the goal of this function is to generate a neighbor solution while trying to reduce violations
def tweak_findValid(old_sol, max_batch=5):
    global tabu_list
    new_sol = old_sol.copy()
    violations = np.zeros(NUM_KNAPSACKS)        #violations[k] <= 0 -> knap k is feasible, otherwise it violates some contstraint
    for k in range(NUM_KNAPSACKS):              #this cycle computes violations for each knap.
        assigned = np.where(old_sol[0] == k)[0]
        if assigned.size > 0:
            weights = WEIGHTS[assigned].sum(axis=0)
            violations[k] = np.sum(np.maximum(0, weights - CONSTRAINTS[k]))
    if np.max(violations) == 0:                 #if the solution is already ok, do a random tweak (to keep exploring)
        i = randint(0, NUM_ITEMS-1)
        new_sol[0, i] = randint(-1, NUM_KNAPSACKS-1)
        return new_sol
    k = np.argmax(violations)                   #finds the "worst" knap.
    assigned_indices = np.where(old_sol[0] == k)[0]
    items = []
    for i in assigned_indices:
        if i not in tabu_list:                  #keeps only items that are not in the tabu list
            items.append(i)
    if not items:
        items = np.where(old_sol[0] == k)[0]
    item_contrib = []                           #keeps the contribution to the overload for each item (ChatGPT helped here)
    for i in items:
        assigned = np.where(old_sol[0] == k)[0]
        total_weights = WEIGHTS[assigned].sum(axis=0)
        over = np.maximum(0, total_weights - CONSTRAINTS[k])
        contrib = np.sum(np.minimum(over, WEIGHTS[i]))
        item_contrib.append((i, contrib))
    item_contrib.sort(key=lambda x: x[1], reverse=True)     #picks the "worst" items
    batch_size = min(len(item_contrib), randint(1, max_batch))
    items_to_move = [i for i, _ in item_contrib[:batch_size]]
    for item in items_to_move:                              #reassigns the bad items
        best_k = -1
        min_overload = np.inf
        for kk in range(NUM_KNAPSACKS):
            if kk == k:                                     #skips the knap. I'm currently trying to fix
                continue
            assigned_indices = np.where(new_sol[0] == kk)[0]
            current_weights = WEIGHTS[assigned_indices].sum(axis=0) if assigned_indices.size > 0 else np.zeros(NUM_DIMENSIONS)
            potential_weights = current_weights + WEIGHTS[item]
            overload = np.sum(np.maximum(0, potential_weights - CONSTRAINTS[kk]) / (CONSTRAINTS[kk] + 1e-6))
            if overload < min_overload:
                min_overload = overload
                best_k = kk
        if min_overload == 0:
            new_sol[0, item] = best_k
        else:
            new_sol[0, item] = -1
        tabu_list.append(item)                              #adds each moved item to the list so it won't be touched again soon
        if len(tabu_list) > TABU_SIZE:
            tabu_list.pop(0)
    return new_sol

#the goal of this function is to generate a neighbor solution while trying to increase the score
def tweak_findBest(old_sol):
    new_sol = old_sol.copy()
    k = randint(0, NUM_KNAPSACKS-1)                 #picks a random knap.
    assigned = np.where(new_sol[0] == k)[0]         #indices of all items currently in k
    unassigned = np.where(new_sol[0] == -1)[0]      #indices of all unassigned items
    if assigned.size > 0 and unassigned.size > 0:   #keeps going until k isn't empty and there are still some unassigned items
        low_ind= assigned[np.argmin(VALUES[assigned])]         #index of the lowest value item in k
        high_ind = unassigned[np.argmax(VALUES[unassigned])]    #index of the highest value unassigned item
        trial_sol = new_sol.copy()
        trial_sol[0, low_ind] = -1
        trial_sol[0, high_ind] = k
        valid, _, _ = check_sol(trial_sol)
        if valid:                                   #tries to add a good item and remove a bad one and then checks if this way the sol. is still valid
            new_sol = trial_sol
    if unassigned.size > 0 and random() < 0.3:      #with 30% prob. tries to add an item instead of swapping (to keep exploring)
        high_ind = unassigned[np.argmax(VALUES[unassigned])]
        for kk in range(NUM_KNAPSACKS):
            trial_sol = new_sol.copy()
            trial_sol[0, high_ind] = kk
            valid, _, _ = check_sol(trial_sol)
            if valid:
                new_sol = trial_sol
                break
    return new_sol

In [128]:
#simulated annealing with two phases

curr_sol = initial_solution()
valid, _, curr_cost = check_sol(curr_sol)
best_sol = curr_sol.copy()
best_cost = curr_cost

#phase 1: looking for a valid solution
print("----- Phase 1: validity search -----")
temp = 1.0                                  #initial temperature
max_i= 20000                                #maximum number of iterations for this phase
for iteration in range(max_i):
    new_sol = tweak_findValid(curr_sol)
    valid, _, new_cost = check_sol(new_sol)
    diff = new_cost - curr_cost             #difference from new to old sol. (if <0, new sol. is better)
    p = np.exp(-diff / temp)                #acceptance probability (simulated annealing)
    if diff <= 0 or random() < p:           #with probability p I can still accept a worse sol.
        curr_sol = new_sol
        curr_cost = new_cost
        if curr_cost < best_cost:           #keeps track of the best sol. so far
            best_cost = curr_cost
            best_sol = curr_sol.copy()
    temp *= 0.999                           #lowers the temperature a little every iteration
    if valid and curr_cost == 0:
        print(f"Valid solution found at iteration {iteration}!")
        break

#phase 2: looking for the highest score solution
print("----- Phase 2: score maximization search -----")
max_i = 5500
for iteration in range(max_i):
    new_sol = tweak_findBest(curr_sol)
    valid, score, _ = check_sol(new_sol)
    if valid and score > np.sum(VALUES[curr_sol[0] != -1]):     #checks if the new sol. is valid and has a tot. score higher then the current sol.
        curr_sol = new_sol
        best_sol = new_sol.copy()
    elif valid and random() < 0.05:                             #occasionally accepts new solutions that aren't better to explore
        curr_sol = new_sol
    if iteration % 250 == 0:                                    #shows updates in the search
        assigned_value = np.sum(VALUES[curr_sol[0] != -1])
        assigned_items = np.sum(curr_sol[0] != -1)
        print(f"Iter {iteration:6d} | Assigned items: {assigned_items} | Total value: {assigned_value}")

#final solution
valid, score, _ = check_sol(best_sol)
assigned_items = np.sum(best_sol[0] != -1)
total_value = np.sum(VALUES[best_sol[0] != -1])
print(f"\nFinal solution: {assigned_items} items assigned, total value = {total_value}, valid = {valid}")

#function that returns the tot. value of the assigned items as a % of the theoretical maximum (if all items were taken)
def value_percentage(solution):
    assigned_value = np.sum(VALUES[solution[0] != -1])
    max_possible_value = np.sum(VALUES)
    perc = (assigned_value / max_possible_value) * 100
    return perc

perc = value_percentage(best_sol)
print(f"Total value assigned: {perc:.2f}% of the theoretical maximum")



----- Phase 1: validity search -----
Valid solution found at iteration 1!
----- Phase 2: score maximization search -----
Iter      0 | Assigned items: 2404 | Total value: 1202793
Iter    250 | Assigned items: 2422 | Total value: 1257806
Iter    500 | Assigned items: 2439 | Total value: 1312214
Iter    750 | Assigned items: 2452 | Total value: 1347416
Iter   1000 | Assigned items: 2465 | Total value: 1382710
Iter   1250 | Assigned items: 2483 | Total value: 1419284
Iter   1500 | Assigned items: 2490 | Total value: 1443078
Iter   1750 | Assigned items: 2493 | Total value: 1458273
Iter   2000 | Assigned items: 2505 | Total value: 1486459
Iter   2250 | Assigned items: 2512 | Total value: 1504791
Iter   2500 | Assigned items: 2519 | Total value: 1526187
Iter   2750 | Assigned items: 2525 | Total value: 1547487
Iter   3000 | Assigned items: 2531 | Total value: 1560233
Iter   3250 | Assigned items: 2539 | Total value: 1578929
Iter   3500 | Assigned items: 2543 | Total value: 1593180
Iter   37